In [65]:
#import libraries

import pandas as pd
import requests
import credentials

from nltk.sentiment import SentimentIntensityAnalyzer

In [66]:
key = credentials.youtube

In [67]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [68]:
url_comments = f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=1000&videoId=KHpMxHOGelo&key={key}'
comment_api = requests.get(url_comments).json()

In [69]:
comment = comment_api['items'][0]['snippet']['topLevelComment']['snippet']['textDisplay']

In [70]:
def get_sentiment(df) :

    for comments in comment_api['items'] :
        video_id = 'KHpMxHOGelo'

        comment = comments['snippet']['topLevelComment']['snippet']['textDisplay']
        senti = sentiment_analyzer.polarity_scores(comment)

        temp_df = pd.DataFrame({
            'video_id' : [video_id],
            'pos' : [senti['pos']],
            'neu' : [senti['neu']],
            'neg' : [senti['neg']]})

        df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [71]:
df = pd.DataFrame(columns=['video_id','pos','neu','neg'])

main_df = get_sentiment(df)

In [72]:
main_df

,video_id,pos,neu,neg
0,KHpMxHOGelo,0.394,0.606,0.000
1,KHpMxHOGelo,0.000,1.000,0.000
2,KHpMxHOGelo,0.000,1.000,0.000
3,KHpMxHOGelo,0.000,0.000,0.000
4,KHpMxHOGelo,0.404,0.596,0.000
...,...,...,...,...
93,KHpMxHOGelo,0.000,1.000,0.000
94,KHpMxHOGelo,0.000,0.345,0.655
95,KHpMxHOGelo,0.000,1.000,0.000
96,KHpMxHOGelo,0.000,1.000,0.000


In [73]:
avg_main = main_df.groupby(['video_id']).mean().round(2)

In [74]:
avg_main

,pos,neu,neg
video_id,,,
KHpMxHOGelo,0.15,0.77,0.07
